# Day 17
## Part 1


In [2]:
import parse

def parse_data(s):
    r = parse.parse("""Register A: {a:d}
Register B: {b:d}
Register C: {c:d}

Program: {program}""", s)
    return r["a"], r["b"], r["c"], [int(x) for x in r["program"].split(",")]

test_data = parse_data("""Register A: 729
Register B: 0
Register C: 0

Program: 0,1,5,4,3,0""")
test_data

(729, 0, 0, [0, 1, 5, 4, 3, 0])

In [3]:
def combo(x, a, b, c):
    match x:
        case 0 | 1 | 2 | 3:
            return x
        case 4:
            return a
        case 5:
            return b
        case 6:
            return c

def run(a, b, c, program):
    i = 0
    while i < len(program):
        lit = program[i+1]
        com = combo(lit, a, b, c)
        match program[i]:
            case 0:
                a = a // (2 ** com)
            case 1:
                b = b ^ lit
            case 2:
                b = com % 8
            case 3:
                if a != 0:
                    i = lit - 2
            case 4:
                b = b ^ c
            case 5:
                yield com % 8
            case 6:
                b = a // (2 ** com)
            case 7:
                c = a // (2 ** com)
        i += 2

def part_1(data):
    return ",".join(str(x) for x in run(*data))

part_1(test_data)

'4,6,3,5,6,3,5,2,1,0'

In [4]:
data = parse_data(open("input").read())
part_1(data)

'1,7,6,5,1,0,5,0,7'

## Part 2

I'm going to have to think about this one.

The code for my input can be translated to the following Python function.

In [34]:
a, b, c, _ = data

def run_2(a, b, c):
    result = []
    while a > 0:
        # Takes the lowest 3 bits of a and 
        # converts
        b = (a % 8) ^ 3
        # Take all but the lowest b bits of a
        c = a // (2**b)
        # Another less predictable conversion
        b = b ^ c
        # Strip the last 3 bits from a
        a = a // 8
        b = b ^ 5
        result.append(b % 8)
    return result
    
run_2(a, b, c)

[1, 7, 6, 5, 1, 0, 5, 0, 7]

I think this can be built backwards. For each value work out what the next one is based on previous values.

In [50]:
result = 0
for i, x in enumerate(reversed(program)):
    result = result * 8
    required_so_far = program[-i:]
    print(required_so_far)
    for a in range(8):
        if run_2(result + a, 0, 0) == required_so_far:
            print(a)
            result += a
            break
run_2(result, 0, 0)

[2, 4, 1, 3, 7, 5, 4, 2, 0, 3, 1, 5, 5, 5, 3, 0]
[0]
6
[3, 0]
1
[5, 3, 0]
1
[5, 5, 3, 0]
1
[5, 5, 5, 3, 0]
1
[1, 5, 5, 5, 3, 0]
2
[3, 1, 5, 5, 5, 3, 0]
1
[0, 3, 1, 5, 5, 5, 3, 0]
[2, 0, 3, 1, 5, 5, 5, 3, 0]
[4, 2, 0, 3, 1, 5, 5, 5, 3, 0]
[5, 4, 2, 0, 3, 1, 5, 5, 5, 3, 0]
[7, 5, 4, 2, 0, 3, 1, 5, 5, 5, 3, 0]
[3, 7, 5, 4, 2, 0, 3, 1, 5, 5, 5, 3, 0]
[1, 3, 7, 5, 4, 2, 0, 3, 1, 5, 5, 5, 3, 0]
[4, 1, 3, 7, 5, 4, 2, 0, 3, 1, 5, 5, 5, 3, 0]


[6, 6, 6, 6, 6, 6, 6, 7, 3, 1, 5, 5, 5, 3, 0]

Hmm, halfway through it can't find a value based on the previous choices. I'm going to have to search through the possibilities, I think. (Unless there's an analytical method I'm missing.)